In [12]:
import os
import re

def get_synched_frames(scene_dir):
    rgb_images = {}
    frame_list = {}
    accel_recs = {}

    files = sorted(os.listdir(scene_dir))

    num_depth = 0
    num_rgb = 0
    num_accel = 0

    for file_name in files:
        if re.match(r'^d-*', file_name):
            num_depth += 1
            frame_list[num_depth] = {'rawDepthFilename': file_name}
        elif re.match(r'^r-*', file_name):
            num_rgb += 1
            rgb_images[num_rgb] = {'name': file_name}
        elif re.match(r'^a-*', file_name):
            num_accel += 1
            accel_recs[num_accel] = {'name': file_name}

    print(f'Found {num_depth} depth, {num_rgb} rgb images, and {num_accel} accel dumps.')

    jj = 1  # Current RGB pointer.
    kk = 1  # Current Accel pointer.

    for ii in range(1, num_depth+1):
        print(f'Matching depth image {ii}/{num_depth}')

        # Parse the timestamp.
        time_parts_depth = frame_list[ii]['rawDepthFilename'][2:].split('-')
        time_parts_rgb = rgb_images[jj]['name'][2:].split('-')
        time_parts_accel = accel_recs[kk]['name'][2:].split('-')

        t_depth = float(time_parts_depth[0])
        t_rgb = float(time_parts_rgb[0])
        t_accel = float(time_parts_accel[0])

        t_diff = abs(t_depth - t_rgb)
        # Advance the curInd until the difference in times gets worse.
        while jj < num_rgb:
            time_parts_rgb = rgb_images[jj+1]['name'][2:].split('-')
            t_rgb = float(time_parts_rgb[0])

            tmp_diff = abs(t_depth - t_rgb)
            if tmp_diff > t_diff:
                break
            t_diff = tmp_diff

            # Otherwise, it's better! and we should update jj
            jj += 1

        # ACCEL
        t_diff = abs(t_depth - t_accel)
        # Advance the curInd until the difference in times gets worse.
        while kk < num_accel:
            time_parts_accel = accel_recs[kk+1]['name'][2:].split('-')
            t_accel = float(time_parts_accel[0])

            tmp_diff = abs(t_depth - t_accel)
            if tmp_diff > t_diff:
                break
            t_diff = tmp_diff

            # Otherwise, it's better! and we should update kk
            kk += 1

        print(f'Matched depth {ii} to rgb {jj} and accel {kk}.')

        # Now save the current RGB filename and ACCEL filename.
        frame_list[ii]['rawRgbFilename'] = rgb_images[jj]['name']
        frame_list[ii]['accelFilename'] = accel_recs[kk]['name']

    return frame_list

# Define the scene directory path
scene_dir = '/Users/ashwinkashyap/Downloads/reception_rooms/reception_room_0001a'

# Call the get_synched_frames function
frame_list = get_synched_frames(scene_dir)

# Access the synchronized frames from frame_list
for depth_frame in frame_list.values():
    raw_depth_filename = depth_frame['rawDepthFilename']
    raw_rgb_filename = depth_frame['rawRgbFilename']
    accel_filename = depth_frame['accelFilename']
    print(f'Depth frame: {raw_depth_filename}, RGB frame: {raw_rgb_filename}, Accel file: {accel_filename}')

Found 856 depth, 501 rgb images, and 2029 accel dumps.
Matching depth image 1/856
Matched depth 1 to rgb 1 and accel 1.
Matching depth image 2/856
Matched depth 2 to rgb 1 and accel 3.
Matching depth image 3/856
Matched depth 3 to rgb 1 and accel 5.
Matching depth image 4/856
Matched depth 4 to rgb 1 and accel 7.
Matching depth image 5/856
Matched depth 5 to rgb 1 and accel 9.
Matching depth image 6/856
Matched depth 6 to rgb 1 and accel 11.
Matching depth image 7/856
Matched depth 7 to rgb 1 and accel 13.
Matching depth image 8/856
Matched depth 8 to rgb 1 and accel 15.
Matching depth image 9/856
Matched depth 9 to rgb 1 and accel 17.
Matching depth image 10/856
Matched depth 10 to rgb 1 and accel 19.
Matching depth image 11/856
Matched depth 11 to rgb 1 and accel 21.
Matching depth image 12/856
Matched depth 12 to rgb 1 and accel 24.
Matching depth image 13/856
Matched depth 13 to rgb 1 and accel 26.
Matching depth image 14/856
Matched depth 14 to rgb 1 and accel 28.
Matching depth i

In [18]:
import os
import re

def get_synched_frames(scene_dir):
    rgb_images = []
    frame_list = []

    # List all files in the scene directory
    files = sorted(os.listdir(scene_dir))

    # Filter and process RGB and depth images
    for file_name in files:
        if re.match(r'^d-*', file_name):
            frame_list.append({'rawDepthFilename': file_name})
        elif re.match(r'^r-*', file_name):
            rgb_images.append({'name': file_name})

    num_depth = len(frame_list)
    num_rgb = len(rgb_images)
    print(f'Found {num_depth} depth, {num_rgb} rgb images')

    # Synchronize RGB and depth images
    for i, frame in enumerate(frame_list):
        depth_parts = re.split(r'-', frame['rawDepthFilename'][2:])
        depth_time = float(depth_parts[0])

        best_match = None
        min_time_diff = float('inf')

        for rgb_image in rgb_images:
            rgb_parts = re.split(r'-', rgb_image['name'][2:])
            rgb_time = float(rgb_parts[0])

            time_diff = abs(depth_time - rgb_time)

            if time_diff < min_time_diff:
                min_time_diff = time_diff
                best_match = rgb_image

        if best_match:
            frame['rawRgbFilename'] = best_match['name']

    return frame_list

# Specify the scene directory path
scene_dir = '/Users/ashwinkashyap/Downloads/reception_rooms/reception_room_0001a'

# Call the get_synched_frames function
frame_list = get_synched_frames(scene_dir)

# Print the synchronized frames
for frame in frame_list:
    print(f"Matched depth {frame['rawDepthFilename']} to rgb {frame['rawRgbFilename']}")


Found 856 depth, 501 rgb images
Matched depth d-1315418947.517688-2288425273.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.549722-2290427428.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.581713-2292429583.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.613756-2294431738.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.645760-2296433893.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.677791-2298436048.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.709784-2300438203.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.741804-2302440358.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.773816-2304442513.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.805818-2306444668.pgm to rgb r-1315418959.350702-2998682927.ppm
Matched depth d-1315418947.837806-2308446823.pgm to rgb r-